In [1]:
# Import libraries
from recordlinkage import preprocessing, Compare
from recordlinkage import index
import os
import pandas as pd
import pathlib

os.chdir('..')

from db_manager import DBManager

In [2]:
project_dir = os.getcwd()

In [3]:
# Get data
db_authors = DBManager('bioinfo_authors')

In [4]:
# Process data
authors_db = db_authors.search({})
authors = []
for author_db in authors_db:
    # Clean the author's name
    s_author = pd.Series(author_db['name'])
    clean_author = preprocessing.clean(s_author, strip_accents='unicode')
    author_str = clean_author.to_string(header=False, index=False).strip()
    author_str_arr = author_str.split()
    authors.append(
        {
            'first_name': ' '.join(author_str_arr[:len(author_str_arr)-1]),
            'last_name': author_str_arr[-1],
            'sex': author_db['gender']
        }
    )

In [8]:
# Create blocks
import recordlinkage
authors_pd = pd.DataFrame(authors)
indexer = index.SortedNeighbourhoodIndex('last_name', window=9)
candidate_links = indexer.index(authors_pd)

/home/jorge/python_venvs/gender_identification/lib/python3.6/site-packages/recordlinkage/index.py:348: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  verify_integrity=False)
/home/jorge/python_venvs/gender_identification/lib/python3.6/site-packages/recordlinkage/base.py:288: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  pairs = pairs[pairs.labels[0] > pairs.labels[1]]


In [12]:
# Compare records and find duplicates in blocks
compare_cl = Compare()
compare_cl.string('first_name', 'first_name', method='jarowinkler', threshold=0.95, label='first_name')
compare_cl.string('last_name', 'last_name', method='jarowinkler', threshold=0.95, label='last_name')
compare_cl.exact('sex', 'sex', label='sex')
features = compare_cl.compute(candidate_links, authors_pd)

In [13]:
features.head(10)

,,first_name,last_name,sex
99825,99012,0.0,0.0,0
101903,99012,0.0,0.0,1
102924,99012,0.0,0.0,1
104243,99012,0.0,0.0,0
105062,99012,0.0,0.0,1
108713,99012,0.0,0.0,1
28708,18432,0.0,0.0,1
28871,18432,0.0,0.0,1
29250,18432,0.0,0.0,0
39391,18432,0.0,0.0,1


In [14]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0       4693
2.0    2941873
1.0    3704952
0.0     894712
dtype: int64

In [15]:
# Get duplicates by matching records that have similar first_name, last_name, and sex
duplicates = features[features.sum(axis=1) > 2]

In [16]:
duplicates.head()

,,first_name,last_name,sex
6915,6002,1.0,1.0,1
120099,12205,1.0,1.0,1
16191,5057,1.0,1.0,1
26790,5057,1.0,1.0,1
104193,103819,1.0,1.0,1


In [17]:
authors_pd.loc[[6915, 6002]]

,first_name,last_name,sex
6915,manuel,campos,male
6002,manel,camps,male


In [18]:
authors_pd.loc[[120099, 12205]]

,first_name,last_name,sex
120099,christian d,lorenz,male
12205,christian,lorenzi,male


In [19]:
authors_pd.loc[[16191, 5057]]

,first_name,last_name,sex
16191,erik m van,mulligen,male
5057,erik m van,mullingen,male


In [20]:
authors_pd.loc[[26790, 5057]]

,first_name,last_name,sex
26790,erik van,mulligen,male
5057,erik m van,mullingen,male


In [33]:
# store results in a new dataframe
dup_df = pd.DataFrame(columns=['1_first_name', '1_last_name', '1_sex', '2_first_name', '2_last_name', '2_sex'])
pairs_exactly_equal = 0
for i in range(0, len(duplicates)):
    columns_dup1 = ['1_'+col_name for col_name in list(authors_pd.loc[duplicates.iloc[i].name[0]].index)]
    first_name_1, last_name_1, sex_1 = authors_pd.loc[duplicates.iloc[i].name[0]]['first_name'], \
                                       authors_pd.loc[duplicates.iloc[i].name[0]]['last_name'], \
                                       authors_pd.loc[duplicates.iloc[i].name[0]]['sex']
    dup1 = pd.DataFrame([authors_pd.loc[duplicates.iloc[i].name[0]]], 
                         columns=list(authors_pd.loc[duplicates.iloc[i].name[0]].index),
                         index=[i])
    dup1.columns = columns_dup1
    columns_dup2 = ['2_'+col_name for col_name in list(authors_pd.loc[duplicates.iloc[i].name[1]].index)]
    first_name_2, last_name_2, sex_2 = authors_pd.loc[duplicates.iloc[i].name[1]]['first_name'], \
                                       authors_pd.loc[duplicates.iloc[i].name[1]]['last_name'], \
                                       authors_pd.loc[duplicates.iloc[i].name[1]]['sex']
    dup2 = pd.DataFrame([authors_pd.loc[duplicates.iloc[i].name[1]]], 
                         columns=list(authors_pd.loc[duplicates.iloc[i].name[1]].index),
                         index=[i])
    dup2.columns = columns_dup2
    if first_name_1 == first_name_2 and last_name_1 == last_name_2 and sex_1 == sex_2:
        pairs_exactly_equal += 1
        continue
    dup_df = dup_df.append(pd.concat([dup1, dup2], axis=1), ignore_index=True)
print(f"Found {pairs_exactly_equal} pairs that are exactly equal")

Found 1075 pairs that are exactly equal


In [35]:
print(f"Found {dup_df.shape[0]} records that are potentially duplicates")

Found 3618 records that are potentially duplicates


In [36]:
dup_df.head(10)

,1_first_name,1_last_name,1_sex,2_first_name,2_last_name,2_sex
0,manuel,campos,male,manel,camps,male
1,christian d,lorenz,male,christian,lorenzi,male
2,erik m van,mulligen,male,erik m van,mullingen,male
3,erik van,mulligen,male,erik m van,mullingen,male
4,michael,rothe,male,michael,rother,male
5,hua,chen,unknown,hua,cheng,unknown
6,li,chen,unknown,li,cheng,unknown
7,chia ling,chen,unknown,chia ying,cheng,unknown
8,shiyuan,chen,unknown,shi yuan,cheng,unknown
9,chao,chen,unknown,chao,cheng,unknown


In [37]:
dup_df.to_csv(pathlib.Path(project_dir, 'data/potential_duplicates.csv').as_posix())